# OMI_01_timeseries_mat.m

In [1]:
import numpy as np

In [2]:
import h5py

In [2]:
### Package Import
import sys
import os
base_dir = os.environ['GEMS_HOME']
project_path = os.path.join(base_dir, 'python-refactor')
sys.path.insert(0, project_path)

from Code.utils import matlab
#from Code.utils import helpers

import scipy.io as sio
import numpy as np
import glob
import time
from scipy.spatial import Delaunay

In [72]:
### Package Import
import sys
import os
base_dir = os.environ['GEMS_HOME']
project_path = os.path.join(base_dir, 'python-refactor')
sys.path.insert(0, project_path)

from Code.utils import matlab
#from Code.utils import helpers

import scipy.io as sio
import numpy as np
import glob
import time
from scipy.spatial import Delaunay
import h5py 

### Setting path
omi_path = os.path.join(project_path, 'Data', 'Raw', 'OMI')
data_path = os.path.join(project_path, 'Data',)
# addpath(genpath([path_data,'/matlab_func/']))                     % Add the path of external function (matlab_func) folder with subfolders

### Setting period
YEARS = [2016] #, 2018, 2019

def create_large_array(filename, size, isFillNan=True):
    data = np.memmap(write_path, dtype='float64', mode='w+', shape=(1036800,days))
    if isFillNan:
        data[:] = np.nan
    return data

### OMNO2d
for yr in YEARS:
    curr_path = os.path.join(omi_path, 'L3', 'OMNO2d', str(yr))
    list_char = glob.glob(os.path.join(curr_path, '*.he5'))
    list_char = [os.path.basename(f) for f in list_char]
    doy_000 = matlab.datenum(f'{yr}0000')
    list_doy = [matlab.datenum(f'{x[19:23]}{x[24:28]}')-doy_000 for x in list_char]
    if yr%4==0:
        days = 366
    else:
        days = 365
        
    write_path = os.path.join(data_path, 'Preprocessed_raw', 'OMI_tempConv', f'OMNO2d_{yr}_DU.mat')
    matlab.check_make_dir(os.path.join(data_path, 'Preprocessed_raw', 'OMI_tempConv')) # For debugging
    size = (1036800, days)
    with h5py.File(write_path, 'w') as f:
        data_yr = f.create_dataset('data_yr', shape=size, fillvalue=np.nan)
        for i in range(len(list_char)):
            data = matlab.h5read(os.path.join(curr_path, list_char[i]), 
                                 '/HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2TropCloudScreened')
            data = np.float64(data); 
            data[data<=-1.2676506e+30] = np.nan # Assign NaN value to pixel that is out of valid range
            data = data * 3.7216e-17                                   # molec/cm2 to DU
            data_yr[:, list_doy[i]] = data.flatten() # Put daily data as column vector
            print (f'OMNO2d_{yr}_{list_doy[i]:03d}')
        np.where(np.isnan(data_yr), -9999, data_yr) # data_yr(isnan(data_yr))=-9999;
        # header = [f'd{x:03d}' for x in range(1, days+1)] # Not used after
    break

ValueError: Index (366) out of range (0-365)

In [64]:
def create_large_array(path, dataset, size, isFillNan=True):
    with h5py.File(path, 'w') as f:
        if isFillNan:
            dst = f.create_dataset(dataset, shape=size, fillvalue=np.nan)
        else:
            dst = f.create_dataset(dataset, shape=size, dtype=np.float64)
    return dst

In [23]:
import h5py
h5file = h5py.File(fname, 'w')

In [24]:
h5file.create_dataset('data_yr', shape=(1036800, days), dtype=np.float64)

<HDF5 dataset "data_yr": shape (1036800, 366), type "<f8">

In [59]:
!rm test.mat

In [60]:
h5file = h5py.File(fname, 'w')

In [61]:
dst = h5file.create_dataset("myvideo", shape=(1036800, days),
                           fillvalue=np.nan)

In [63]:
dst[0]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [78]:
np.random.randint(10)

6

In [68]:
!rm test.mat

In [69]:
import h5py
import numpy as np

# create an hdf5 file
with h5py.File("test.mat") as f:
    # create a dataset for your movie
    dst = f.create_dataset("myvideo", shape=(10, 200, 200, 3),
                           dtype=np.uint8)
    # fill the 10 frames with a random image
    for frame in range(10):
        dst[frame] = np.random.randint(255, size=(200, 200, 3))

/home/sehyun/GEMS_python/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """


In [66]:
h5file.close()

In [61]:
fname

'test.mat'

In [75]:
!ls videos.mat -sh

1.2M videos.mat


In [ ]:

### OMSO2e
for yr in YEARS:
    os.chdir(os.path.join(omi_path, 'L3_grid', 'OMSO2e', str(yr)))
    list_char = matlab.dir('.', '.he5')
    doy_000 = datenum(f'{yr}0000')
    list_doy = map(lambda x: x[19:23]+x[23:28]-doy_000, list_char)
    if yr%4==0:
        days = 366
    else:
        days = 365
    
    data_yr = np.full(1036800, np.nan)
    for i in range(len(list_char)):
        data = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/ColumnAmountSO2_PBL')
        rcf = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/RadiativeCloudFraction')
        sza = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/OMI Total Column Amount SO2/Data Fields/SolarZenithAngle')
    
        # Assign NaN value to pixel that is out of valid range
        data = np.float64(data) # data[data<=-1.2676506e+30]=np.nan;
        rcf = np.float64(rcf) # rcf[rcf<=-1.2676506e+30]=np.nan;
        sza = np.float64[sza] # sza[sza<=-1.2676506e+30]=np.nan;
        
        # Filtering SO2_VCD with the condition of radiative cloud fraction and
        # solar zenith angle
        data2 = data;
        data2[rcf>=0.3]=np.nan;
        data2[sza>=78]=np.nan;
        
        data_yr[, list_doy(i)] = data2.flatten() # Put daily data as column vector
        print (f'OMSO2e_m_{yr}_{list_doy[i]:03d}')
    data_yr(isnan(data_yr))=-9999;
    prefix = np.tile('d', [days, 1]) % prefix for header
    header = map(lambda x: f'{x:03d}', prefix) # header = cellstr([prefix,num2str([1:days]','%03i')]);
    sio.savemat(os.path.join(data_path, 'OMI_L3_tempConv', f'OMSO2d_{yr}_DU.mat'), mdict={'data_yr':data_yr})

### OMDOAO3e
for yr in YEARS:
    tStart = time.time()
    os.chdir(os.path.join(omi_path, 'L3_grid', 'OMDOAO3e', str(yr)))
    list_char = matlab.dir('.', '.he5')
    doy_000 = datenum(f'{yr}0000')
    list_doy = map(lambda x: x[19:23]+x[23:28]-doy_000, list_char)
    if yr%4==0:
        days = 366
    else:
        days = 365
    
    data_yr = np.full(1036800, np.nan)
    for i in range(len(list_char)):
        data = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/ColumnAmountO3')
        cf = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/CloudFraction')
        sza = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/ColumnAmountO3/Data Fields/SolarZenithAngle')
    
        # Assign NaN value to pixel that is out of valid range
        data = np.float64(data) # data[data<=-1.2676506e+30]=np.nan;
        cf = np.float64(cf) # rcf[rcf<=-1.2676506e+30]=np.nan;
        sza = np.float64[sza] # sza[sza<=-1.2676506e+30]=np.nan;
        
        # Filtering SO2_VCD with the condition of radiative cloud fraction and
        # solar zenith angle
        data2 = data;
        data2[cf>=0.3]=np.nan;
        data2[sza>=78]=np.nan;
        
        data_yr[, list_doy(i)] = data2.flatten() # Put daily data as column vector
        print (f'OMDOAO3e_m_{yr}_{list_doy[i]:03d}')
    data_yr(isnan(data_yr))=-9999;
    prefix = np.tile('d', [days, 1]) % prefix for header
    header = map(lambda x: f'{x:03d}', prefix) # header = cellstr([prefix,num2str([1:days]','%03i')]);
    sio.savemat(os.path.join(data_path, 'OMI_L3_tempConv', f'OMSO2d_{yr}_DU.mat'), mdict={'data_yr':data_yr})
    tElapsed = time.time() - tStart
    print (f'{tElapsed} second')

### OMHCHOG
for yr in YEARS:
    os.chdir(os.path.join(omi_path, 'L2_grid', 'OMHCHOG', str(yr)))
    list_char = matlab.dir('.', '.he5')
    doy_000 = datenum(f'{yr}0000')
    list_doy = map(lambda x: x[19:23]+x[23:28]-doy_000, list_char)
    if yr%4==0:
        days = 366
    else:
        days = 365
    
    data_yr = np.full(1036800, np.nan)
    for i in range(len(list_char)):
        data = mathlab.h5read(list_char[i],'/HDFEOS/GRIDS/OMI Total Column Amount HCHO/Data Fields/ColumnAmountDestriped')
        # Assign NaN value to pixel that is out of valid range
        data = np.float64(matlab.permute(data,[2,1,3]))
        data[data<=-1.0e+30] = np.nan
        VCD = data * 3.7216e-17                                  # molec/cm2 to DU
        
        QA = matlab.h5read(list_char[i],'/HDFEOS/GRIDS/OMI Total Column Amount HCHO/Data Fields/MainDataQualityFlag')
        QA = np.float64(permute(QA, [2,1,3]))
        QA[QA<=-30000] = np.nan
        
        sza = matlab.h5read(list_char[i],'/HDFEOS/GRIDS/OMI Total Column Amount HCHO/Data Fields/SolarZenithAngle')
        sza = np.float64(matlab.permute(sza,[2,1,3]))
        sza[sza<=-1.0e+30] = np.nan
        
        VCD_filtered = np.zeros(VCD.shape)
        for k in range(15):
            VCD_temp = VCD[:,:,k]
            QA_temp = QA[:,:,k]
            sza_temp = sza[:,:,k]
            
            VCD_temp[QA_temp!=0] = np.nan #  Good columns (0) / Suspect columns (1) / Bad columns (2) / missing (<0)
            VCD_temp[sza_temp>88] = np.nan
            VCD_filtered[:,:,k] = VCD_temp
        VCD_avg = np.nanmean(VCD_filtered, axis=2)
        data_yr[, list_doy(i)] = VCD_avg.flatten() # Put daily data as column vector
        print (f'OMHCHOG_{yr}_{list_doy[i]:03d}')
        
        if list_doy[i]%50==0:
            data_sub = data_yr[:, list_doy[i]-48:list_doy[i]]
            sio.savemat(os.path.join(data_path, 'pre', 'OMI_L3_tempConv', f'OMHCHOG_{yr}_{list_doy[i]:03d}.mat'), mdict={'data_sub':data_sub})
    # Gathering part files
    data_yr = []
    for m in range(50, 350+1, 50):
        data_sub = sio.loadmat(os.path.join(data_path, 'pre', 'OMI_L3_tempConv', f'OMHCHOG_{yr}_{list_doy[i]:03d}.mat'))
        data_yr = np.concatenate((data_yr, data_sub), axis=1) # data_yr=[data_yr,data_sub];
    data_sub = sio.loadmat(os.path.join(data_path, 'pre', 'OMI_L3_tempConv', f'OMHCHOG_{yr}_end.mat'))
    data_yr = np.concatenate((data_yr, data_sub), axis=1)
        
    data_yr(isnan(data_yr))=-9999;
    prefix = np.tile('d', [days, 1]) % prefix for header
    header = map(lambda x: f'{x:03d}', prefix) # header = cellstr([prefix,num2str([1:days]','%03i')]);
    sio.savemat(os.path.join(data_path, 'OMI_L3_tempConv', f'OMHCHOG_{yr}_DU.mat'), mdict={'data_yr':data_yr})

# OMI_02_temporal_convolution_with_gaussian

In [ ]:
### Package import
import os
import h5py
import time
import numpy as np

### Common
import sys
project_path = 'C:\\Users\\user\\Downloads\\matlab2python\\matlab2python\\python-refactor'
#project_path = '/home/cogito/Uncertainty/matlab2python/python-refactor'
sys.path.insert(0, project_path)
from Code.utils import matlab
from Code.utils import helpers

### Setting path
data_path = os.path.join(project_path, 'Data')
work_path = os.path.join(data_path, 'pre', 'OMI_L3_tempConv')
# addpath(genpath([path_data,'/matlab_func/']))  % Add the path of external function (matlab_func) folder with subfolders

pname_list = ['OMNO2d','OMSO2e_m','OMDOAO3e_m','OMHCHOG']

mask = np.zeros((720, 1440))
mask[340:552, 1020:1308] = 1
mask = mask.flatten()


### Temporal convolution with gaussian
# for i in range(4):
i = 3
pname = pname_list[i]

### Load data
YEARS = range(2005, 2019+1)
for yr in YEARS:
    # data_yr = sio.loadmat(os.path.join(work_path, f'{pname}_{yr}.mat'))
    data_yr = sio.loadmat(os.path.join(work_path, f'{pname}_trop_CS_{yr}_DU.mat'))
    data_subset = data_yr[mask==1, :]
    data = np.concatenate((data, data_subset), axis=1)
data_org = data
data[data==-9999] = np.nan
sigma = 1
data_conv = np.full(data.shape, np.nan)

for k in range(data.shape[0]):
    for t in range(5478): # for 15 years
        w_sum = 0; x_sum = 0;
        for n in range(5478): # for 15 years
            if np.isnan(data[k,n])==0:
                w = np.exp((-(k-n)**2)/(2*sigma**2)) #  calcuate weights using t,n(days) with sigma # (k+1)-(n-1)=k-n
                x = data[k, n]*w
                w_sum = w_sum+w
                x_sum = x_sum+x
        data_conv[k,t] = x_sum/w_sum
    print (k)
# sio.savemat(os.path.join(work_path, f'tempConv_{pname}_sigma_{sigma}_2005_2019.mat'), mdict={'data_conv':data_conv, 'data':data})
sio.savemat(os.path.join(work_path, f'tempConv_{pname}_trop_CS_sigma{sigma}_2005_2019.mat'), mdict={'data_conv':data_conv, 'data':data})